In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.chrome.options import Options

# selenium 속도 향상 위해 불필요한 옵션을 사용하지 않게하는 코드
options = Options()
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 
                                                    'plugins' : 2, 'popups': 2, 'geolocation': 2,
                                                    'notifications' : 2, 'auto_select_certificate': 2,
                                                    'fullscreen' : 2,
                                                    'mouselock' : 2, 'mixed_script': 2, 
                                                    'media_stream' : 2, 'media_stream_mic' : 2,
                                                    'media_stream_camera': 2, 'protocol_handlers' : 2,
                                                    'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2,
                                                    'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2,
                                                    'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2,
                                                    'durable_storage' : 2}}

options.add_experimental_option('prefs', prefs) 
options.add_argument("start-maximized") 
options.add_argument("disable-infobars") 
options.add_argument("--disable-extensions")

In [2]:
import time
import requests
import pickle
from bs4 import BeautifulSoup

## 카테고리 별 게시글 리스트 페이지에서 유저별 id 파싱 (*전체 24개 카테고리)
## 각 페이지별 무한 스크롤 javaScript 제어를 위한 셀레니움 기능 사용
def get_user_list(base_url):
    
#     크롬드라이버 설정
    chromedriver = 'C:/selenium/chromedriver.exe' 
    driver = webdriver.Chrome(chromedriver)
    driver.get(base_url) 
    
    SCROLL_PAUSE_TIME = 15 # 무한스크롤 멈춤 현상 예방
    last_height = driver.execute_script("return document.body.scrollHeight")
    i = 0
    while True:
        # Scroll down to bottom                                                      
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)                                                
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")  
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height            
        new_height = driver.execute_script("return document.body.scrollHeight")
        i+=1

        if i == 700 :                                            
            break
        last_height = new_height
    
    print(i,"회에서 멈춤")
    
    ## 각 페이별 350번 스크롤 후, BeautifulSoup으로 "전체 게시글 url parsing"
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    a_tags = soup.select('#wrapArticle > div.wrap_article_list.\#keyword_related_contents > ul > li > a.link_post')
    
    save_href = []
    for a_tag in a_tags :
        save_href.append(a_tag['href'])
    
    driver.close()
    
    return save_href # 개별 카테고리별 전체 게시글 url 반환

######################### 
# '지구한바퀴_세계여행?q=g','IT_트렌드?q=g,'스타트업_경험담?q=g','육아_이야기?q=g','
## '사진·촬영?q=g','글쓰기_코치?q=g' 제거
category_list = ['쉽게_읽는_역사?q=g','우리집_반려동물?q=g',
                 '사랑·이별?q=g','감성_에세이?q=g']


# 전체 24개 카테고리별로 돌며 게시글 url 파싱하여 pickle로 저장
for category in category_list:
    each_user_id = []
    each_user_id = get_user_list("https://brunch.co.kr/keyword/"+category)
    with open(category[:-4]+'_userId.txt','wb') as f:
        pickle.dump(each_user_id,f)

700 회에서 멈춤
700 회에서 멈춤


WebDriverException: Message: chrome not reachable
  (Session info: chrome=81.0.4044.138)
